In [61]:
import pandas as pd
import numpy as np
import difflib
import os

In [62]:
os.chdir('C:/Cours DTU/1. Model-based Machine Learning/Project/european_vote_analysis')

In [63]:
data_european = pd.read_csv('data/raw/COMEPELDA_aggregate_v1.00.csv')
data_national = pd.read_csv('data/raw/1999-2019_CHES_dataset_means.csv')
country_codes = pd.read_csv('data/raw/countries.csv')
party_names = pd.read_csv('data/raw/partyNames_ncv2.csv')

In [64]:
# Add country codes to national data
data_european = pd.merge(data_european, country_codes, left_on='cName', right_on='Country', how='left')
# drop 2006, 2007, 2013 years due to particular countries elections
data_european = data_european.drop(data_european[data_european['ElYear'].isin([2006, 2007, 2013])].index)

In [65]:
# add country codes to european data
data_national = pd.merge(data_national, country_codes, left_on='country', right_on='ID', how='left')

In [66]:
data_national = pd.merge(data_national, party_names, left_on='party_id', right_on='partyID', how='left')

In [67]:
#changed national and european order
test_data = pd.merge(data_national,data_european, left_on=["Country", 'partyName'], right_on=["cName", 'pName'], how='outer', indicator=True)

In [68]:
year_corr = {1999: 1999, 
 2004: 2002,
 2009: 2006,
 2014: 2014}
eu_unique_columns = ["Abbreviation", "ElYear", "pName"]
data_unique_columns = ["Abbreviation", "year", "partyName"]
parties_data_eu = data_european[eu_unique_columns].drop_duplicates()
parties_data_nat = data_national[data_unique_columns].drop_duplicates()

In [69]:
for row in parties_data_eu.itertuples():
    y = parties_data_nat[(parties_data_nat["Abbreviation"] == row.Abbreviation) & 
                         (parties_data_nat["year"] == year_corr[row.ElYear])]["partyName"].astype(str)
    x = row.pName
    result = difflib.get_close_matches(x, y, n=1)
    if result:
        parties_data_eu.loc[row.Index, "correspondance"] = result[0]
    else: 
        parties_data_eu.loc[row.Index, "correspondance"] = np.nan

In [70]:
parties_data_eu.correspondance.isna().sum()

214

In [71]:
parties_data_eu.to_csv("data/processed/party_correspondance.csv", index=False)